In [1]:
import pandas as pd
import numpy as np
import warnings
from time import time
import joblib

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.experimental import (
    enable_iterative_imputer,
)  # Required for IterativeImputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

# Models
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
)
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Imbalance handling
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

warnings.filterwarnings("ignore")

#### Configs

In [2]:
CONFIG = {
    "test_size": 0.2,
    "random_state": 42,
    "cv_folds": 5,
    "scale_data": True,
    "handle_imbalance": True,
    "imputation_strategy": "iterative",  # 'iterative', 'knn', or 'simple'
    "n_jobs": -1,
}

#### ADVANCED IMPUTATION

In [3]:
class AdvancedImputer:
    """
    Advanced imputation strategies for missing data.

    Strategies:
    1. 'iterative' - MICE (Multivariate Imputation by Chained Equations)
       - Models each feature with missing values as a function of other features
       - Iteratively predicts missing values
       - Best for complex relationships

    2. 'knn' - K-Nearest Neighbors Imputation
       - Fills missing values using k-nearest neighbors
       - Good for preserving local structure

    3. 'simple' - Statistical imputation
       - Uses median for numerical features
       - Fallback option
    """

    def __init__(
        self, strategy="iterative", n_neighbors=5, max_iter=10, random_state=42
    ):
        self.strategy = strategy
        self.n_neighbors = n_neighbors
        self.max_iter = max_iter
        self.random_state = random_state
        self.imputer = None

    def fit(self, X):
        """Fit the imputer on training data"""
        print(f"\n🔧 Fitting {self.strategy.upper()} imputer...")

        if self.strategy == "iterative":
            # MICE - Models each feature as a function of others
            # Uses BayesianRidge as the estimator (fast and robust)
            self.imputer = IterativeImputer(
                estimator=BayesianRidge(),
                max_iter=self.max_iter,
                random_state=self.random_state,
                verbose=0,
            )

        elif self.strategy == "knn":
            # KNN - Uses nearest neighbors to fill missing values
            self.imputer = KNNImputer(
                n_neighbors=self.n_neighbors,
                weights="distance",  # Closer neighbors have more influence
            )

        elif self.strategy == "simple":
            # Simple statistical imputation (fallback)
            self.imputer = SimpleImputer(strategy="median")

        else:
            raise ValueError(f"Unknown imputation strategy: {self.strategy}")

        self.imputer.fit(X)
        print(f"✓ Imputer fitted successfully")
        return self

    def transform(self, X):
        """Transform data using fitted imputer"""
        if self.imputer is None:
            raise ValueError("Imputer must be fitted before transform")

        # Get column names if DataFrame
        columns = X.columns if isinstance(X, pd.DataFrame) else None
        index = X.index if isinstance(X, pd.DataFrame) else None

        # Transform
        X_imputed = self.imputer.transform(X)

        # Convert back to DataFrame if input was DataFrame
        if columns is not None:
            X_imputed = pd.DataFrame(X_imputed, columns=columns, index=index)

        return X_imputed

    def fit_transform(self, X):
        """Fit and transform in one step"""
        self.fit(X)
        return self.transform(X)

In [4]:
def handle_missing_and_invalid_values(
    X, strategy="iterative", fit_imputer=True, imputer=None
):
    """
    Handle missing values, infinities, and invalid data with advanced imputation.

    Args:
        X: Feature matrix (DataFrame)
        strategy: Imputation strategy ('iterative', 'knn', 'simple')
        fit_imputer: Whether to fit a new imputer (True for train, False for test)
        imputer: Pre-fitted imputer (for test data)

    Returns:
        X_clean: Cleaned feature matrix
        imputer: Fitted imputer (if fit_imputer=True)
    """
    print("\n" + "=" * 80)
    print("HANDLING MISSING AND INVALID VALUES")
    print("=" * 80)

    original_shape = X.shape

    # Step 2: Check for missing values
    print("\n2. Checking for missing values...")
    missing_count = X.isnull().sum().sum()
    missing_cols = X.columns[X.isnull().any()].tolist()

    if missing_count > 0:
        print(
            f"   ⚠️  Found {missing_count} missing values in {len(missing_cols)} columns"
        )
        print(
            f"   Missing percentage: {(missing_count / (X.shape[0] * X.shape[1]) * 100):.2f}%"
        )

        # Show top columns with missing values
        missing_per_col = X.isnull().sum().sort_values(ascending=False)
        top_missing = missing_per_col[missing_per_col > 0].head(10)
        print(f"\n   Top columns with missing values:")
        for col, count in top_missing.items():
            pct = (count / len(X)) * 100
            print(f"     • {col}: {count} ({pct:.2f}%)")

        # Step 3: Apply advanced imputation
        print(f"\n3. Applying {strategy.upper()} imputation...")

        if fit_imputer:
            # Fit new imputer on training data
            imputer = AdvancedImputer(
                strategy=strategy, random_state=CONFIG["random_state"]
            )
            X_clean = imputer.fit_transform(X)
        else:
            # Use pre-fitted imputer on test data
            if imputer is None:
                raise ValueError("Imputer must be provided for test data")
            X_clean = imputer.transform(X)

        # Verify no missing values remain
        remaining_missing = (
            pd.DataFrame(X_clean, columns=X.columns).isnull().sum().sum()
        )
        if remaining_missing > 0:
            print(
                f"   ⚠️  WARNING: {remaining_missing} missing values remain after imputation"
            )
        else:
            print(f"   ✓ All missing values imputed successfully")

    else:
        print(f"   ✓ No missing values found")
        X_clean = X

    # Step 4: Final validation
    print("\n4. Final validation...")
    X_clean_df = pd.DataFrame(X_clean, columns=X.columns, index=X.index)

    # Check for any remaining issues
    has_nan = X_clean_df.isnull().any().any()

    if has_nan:
        print(f"   ⚠️  WARNING: Data still contains invalid values!")
        print(f"      NaN: {has_nan}")
        # Emergency fallback - fill with 0
        X_clean_df = X_clean_df.fillna(0).replace([np.inf, -np.inf], 0)
        print(f"   ✓ Applied emergency fallback (fill with 0)")
    else:
        print(f"   ✓ Data is clean and ready for training")

    print(f"\n5. Summary:")
    print(f"   Original shape: {original_shape}")
    print(f"   Final shape: {X_clean_df.shape}")
    print(f"   Features preserved: {X_clean_df.shape[1]}")

    if fit_imputer:
        return X_clean_df, imputer
    else:
        return X_clean_df

#### 1. LOAD AND EXPLORE DATA

In [5]:
def load_and_explore_data(filepath):
    """Load data and show basic statistics"""
    print("=" * 80)
    print("STEP 1: LOADING DATA")
    print("=" * 80)

    df = pd.read_csv(filepath)
    print(f"✓ Dataset loaded: {df.shape}")

    # Check for issues
    missing = df.isnull().sum().sum()
    duplicates = df.duplicated().sum()

    print(f"\nData Quality Check:")
    print(
        f"  Missing values: {missing} ({(missing/(df.shape[0]*df.shape[1])*100):.2f}%)"
    )
    print(f"  Duplicate rows: {duplicates}")

    if missing > 0:
        print(f"  ℹ️  Will apply advanced imputation in next step")

    # Class distribution
    print(f"\nClass Distribution:")
    print(df["result"].value_counts())
    print(f"Phishing ratio: {df['result'].mean():.2%}")

    # Calculate imbalance ratio
    class_counts = df["result"].value_counts()
    imbalance_ratio = class_counts.max() / class_counts.min()
    print(f"Imbalance ratio: {imbalance_ratio:.2f}:1")

    if imbalance_ratio > 1.5:
        print("⚠️  Dataset is imbalanced - will apply SMOTE")

    return df

#### 2. PREPARE DATA

In [6]:
def prepare_data(df, config):
    """Prepare features and labels, handle scaling and imbalance"""
    print("\n" + "=" * 80)
    print("STEP 2: PREPARING DATA")
    print("=" * 80)

    # Separate features and labels
    X = df.drop(["rec_id", "result"], axis=1)
    y = df["result"]

    print(f"\nDataset Info:")
    print(f"  Features: {X.shape[1]}")
    print(f"  Samples: {len(X)}")
    print(f"  Classes: {y.nunique()}")

    # Split data BEFORE imputation (important!)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=config["test_size"],
        random_state=config["random_state"],
        stratify=y,
    )

    print(f"\nData Split:")
    print(f"  Train set: {X_train.shape}")
    print(f"  Test set: {X_test.shape}")

    # Apply advanced imputation
    X_train, imputer = handle_missing_and_invalid_values(
        X_train, strategy=config["imputation_strategy"], fit_imputer=True
    )

    X_test = handle_missing_and_invalid_values(
        X_test,
        strategy=config["imputation_strategy"],
        fit_imputer=False,
        imputer=imputer,
    )

    # Scale data if configured
    scaler = None
    if config["scale_data"]:
        print("\n" + "=" * 80)
        print("FEATURE SCALING")
        print("=" * 80)
        print(f"\n✓ Scaling features with RobustScaler...")
        print(f"  (RobustScaler is robust to outliers using median and IQR)")

        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Convert back to DataFrame
        X_train = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
        X_test = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

        print(f"  ✓ Features scaled successfully")

    # Handle class imbalance if configured
    if config["handle_imbalance"]:
        print("\n" + "=" * 80)
        print("HANDLING CLASS IMBALANCE")
        print("=" * 80)

        class_counts = y_train.value_counts()
        imbalance_ratio = class_counts.max() / class_counts.min()

        print(f"\nCurrent class distribution:")
        print(f"  Class 0 (Legit): {class_counts.get(0, 0):,}")
        print(f"  Class 1 (Phishing): {class_counts.get(1, 0):,}")
        print(f"  Imbalance ratio: {imbalance_ratio:.2f}:1")

        if imbalance_ratio > 1.5:
            print(f"\n✓ Applying SMOTE (Synthetic Minority Over-sampling)...")
            print(f"  (Generates synthetic samples for minority class)")

            original_size = len(X_train)

            smote = SMOTE(random_state=config["random_state"], k_neighbors=5)
            X_train, y_train = smote.fit_resample(X_train, y_train)

            new_counts = y_train.value_counts()
            print(f"\n  Results:")
            print(f"    Before: {original_size:,} samples")
            print(f"    After: {len(X_train):,} samples")
            print(f"    Class 0: {new_counts.get(0, 0):,}")
            print(f"    Class 1: {new_counts.get(1, 0):,}")
            print(f"    New ratio: {new_counts.max() / new_counts.min():.2f}:1")
        else:
            print(f"\n✓ Classes are balanced enough, skipping SMOTE")

    return X_train, X_test, y_train, y_test, scaler, imputer, X.columns

#### 3. DEFINE MODELS

In [7]:
def get_models(config):
    """Define all models to train"""
    print("\n" + "=" * 80)
    print("STEP 3: DEFINING MODELS")
    print("=" * 80)

    models = {
        # Tree-based ensembles (best for this task)
        "Random Forest": RandomForestClassifier(
            n_estimators=200,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=config["random_state"],
            n_jobs=config["n_jobs"],
        ),
        "Extra Trees": ExtraTreesClassifier(
            n_estimators=200,
            max_depth=20,
            min_samples_split=5,
            random_state=config["random_state"],
            n_jobs=config["n_jobs"],
        ),
        "Gradient Boosting": GradientBoostingClassifier(
            n_estimators=200,
            max_depth=10,
            learning_rate=0.1,
            subsample=0.8,
            random_state=config["random_state"],
        ),
        "XGBoost": XGBClassifier(
            n_estimators=200,
            max_depth=10,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=config["random_state"],
            n_jobs=config["n_jobs"],
            eval_metric="logloss",
        ),
        "LightGBM": LGBMClassifier(
            n_estimators=200,
            max_depth=10,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=config["random_state"],
            n_jobs=config["n_jobs"],
            verbose=-1,
        ),
        "CatBoost": CatBoostClassifier(
            iterations=200,
            depth=10,
            learning_rate=0.1,
            random_state=config["random_state"],
            verbose=False,
        ),
        # Linear models
        # "Logistic Regression": LogisticRegression(
        #     max_iter=1000, random_state=config["random_state"], n_jobs=config["n_jobs"]
        # ),
        # Other models
        # "SVC": SVC(kernel="rbf", probability=True, random_state=config["random_state"]),
        "Naive Bayes": GaussianNB(),
        "KNN": KNeighborsClassifier(n_neighbors=5, n_jobs=config["n_jobs"]),
        "Decision Tree": DecisionTreeClassifier(
            max_depth=20, min_samples_split=5, random_state=config["random_state"]
        ),
        "AdaBoost": AdaBoostClassifier(
            n_estimators=100, learning_rate=0.1, random_state=config["random_state"]
        ),
    }

    print(f"✓ Configured {len(models)} models:")
    for name in models.keys():
        print(f"  • {name}")

    return models

#### 4. TRAIN AND EVALUATE MODELS

In [8]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Train and evaluate a single model"""

    # Train
    start_time = time()
    model.fit(X_train, y_train)
    train_time = time() - start_time

    # Predict
    start_time = time()
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    predict_time = time() - start_time

    # Calculate metrics
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "roc_auc": roc_auc_score(y_test, y_proba),
        "train_time": train_time,
        "predict_time": predict_time,
    }

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    metrics["tn"] = cm[0, 0]
    metrics["fp"] = cm[0, 1]
    metrics["fn"] = cm[1, 0]
    metrics["tp"] = cm[1, 1]

    # Error rates
    metrics["fpr"] = (
        cm[0, 1] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) > 0 else 0
    )
    metrics["fnr"] = (
        cm[1, 0] / (cm[1, 0] + cm[1, 1]) if (cm[1, 0] + cm[1, 1]) > 0 else 0
    )

    return metrics, model, y_pred, y_proba


def train_all_models(models, X_train, X_test, y_train, y_test, config):
    """Train all models and collect results"""
    print("\n" + "=" * 80)
    print("STEP 4: TRAINING MODELS")
    print("=" * 80)

    results = {}

    for name, model in models.items():
        print(f"\n{'─' * 80}")
        print(f"Training: {name}")
        print(f"{'─' * 80}")

        try:
            metrics, trained_model, y_pred, y_proba = evaluate_model(
                model, X_train, X_test, y_train, y_test, name
            )

            results[name] = {
                "model": trained_model,
                "metrics": metrics,
                "y_pred": y_pred,
                "y_proba": y_proba,
            }

            # Print results
            print(f"✓ Training completed in {metrics['train_time']:.2f}s")
            print(f"\nPerformance Metrics:")
            print(f"  Accuracy:  {metrics['accuracy']:.4f}")
            print(f"  Precision: {metrics['precision']:.4f}")
            print(f"  Recall:    {metrics['recall']:.4f}")
            print(f"  F1-Score:  {metrics['f1']:.4f}")
            print(f"  ROC-AUC:   {metrics['roc_auc']:.4f}")
            print(f"\nConfusion Matrix:")
            print(f"  TN: {metrics['tn']:,} | FP: {metrics['fp']:,}")
            print(f"  FN: {metrics['fn']:,} | TP: {metrics['tp']:,}")
            print(f"\nError Rates:")
            print(f"  FPR: {metrics['fpr']:.4f} (legit sites flagged as phishing)")
            print(f"  FNR: {metrics['fnr']:.4f} (phishing sites missed)")

        except Exception as e:
            print(f"✗ Training failed: {str(e)}")
            continue

    return results

#### 5. COMPARE MODELS

In [9]:
def compare_models(results):
    """Compare all models and find the best one"""
    print("\n" + "=" * 80)
    print("STEP 5: MODEL COMPARISON")
    print("=" * 80)

    # Create comparison DataFrame
    comparison = []
    for name, result in results.items():
        metrics = result["metrics"]
        comparison.append(
            {
                "Model": name,
                "Accuracy": metrics["accuracy"],
                "Precision": metrics["precision"],
                "Recall": metrics["recall"],
                "F1-Score": metrics["f1"],
                "ROC-AUC": metrics["roc_auc"],
                "Train Time (s)": metrics["train_time"],
                "FPR": metrics["fpr"],
                "FNR": metrics["fnr"],
            }
        )

    df_comparison = pd.DataFrame(comparison)
    df_comparison = df_comparison.sort_values("ROC-AUC", ascending=False)

    print("\n📊 Model Performance Summary (sorted by ROC-AUC):")
    print("─" * 80)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.precision", 4)
    print(df_comparison.to_string(index=False))

    # Find best model
    best_model_name = df_comparison.iloc[0]["Model"]
    best_metrics = results[best_model_name]["metrics"]

    print("\n" + "=" * 80)
    print("🏆 BEST MODEL")
    print("=" * 80)
    print(f"\nModel: {best_model_name}")
    print(f"\nMetrics:")
    print(
        f"  ✓ Accuracy:  {best_metrics['accuracy']:.4f} ({best_metrics['accuracy']*100:.2f}%)"
    )
    print(
        f"  ✓ Precision: {best_metrics['precision']:.4f} ({best_metrics['precision']*100:.2f}%)"
    )
    print(
        f"  ✓ Recall:    {best_metrics['recall']:.4f} ({best_metrics['recall']*100:.2f}%)"
    )
    print(f"  ✓ F1-Score:  {best_metrics['f1']:.4f}")
    print(f"  ✓ ROC-AUC:   {best_metrics['roc_auc']:.4f}")

    print(f"\nConfusion Matrix:")
    print(
        f"  True Negatives:  {best_metrics['tn']:,} (legit sites correctly identified)"
    )
    print(
        f"  False Positives: {best_metrics['fp']:,} (legit sites flagged as phishing)"
    )
    print(f"  False Negatives: {best_metrics['fn']:,} (phishing sites missed) ⚠️")
    print(f"  True Positives:  {best_metrics['tp']:,} (phishing sites caught)")

    print(f"\nError Analysis:")
    print(
        f"  FPR: {best_metrics['fpr']:.4f} ({best_metrics['fpr']*100:.2f}% of legit sites flagged)"
    )
    print(
        f"  FNR: {best_metrics['fnr']:.4f} ({best_metrics['fnr']*100:.2f}% of phishing sites missed)"
    )

    print(f"\nSpeed:")
    print(f"  Training: {best_metrics['train_time']:.2f}s")
    print(f"  Prediction: {best_metrics['predict_time']*1000:.2f}ms")

    return best_model_name, df_comparison

#### 6. SAVE ARTIFACTS

In [ ]:
def save_artifacts(best_model_name, results, scaler, imputer, feature_names):
    """Save model and related artifacts"""
    print("\n" + "=" * 80)
    print("STEP 7: SAVING ARTIFACTS")
    print("=" * 80)

    # Save best model
    best_model = results[best_model_name]["model"]
    joblib.dump(
        best_model,
        "/home/maliha/Programming/dm/Phishing-Website-Classifier/training/utilities/webcode/best_phishing_model_webcode.pkl",
    )
    print(f"✓ Saved best model: best_phishing_model_webcode.pkl")

    # Save scaler
    if scaler is not None:
        joblib.dump(
            scaler,
            "/home/maliha/Programming/dm/Phishing-Website-Classifier/training/utilities/webcode/feature_scaler_webcode.pkl",
        )
        print(f"✓ Saved scaler: feature_scaler_webcode.pkl")
    # Save imputer (IMPORTANT!)
    if imputer is not None:
        joblib.dump(
            imputer,
            "/home/maliha/Programming/dm/Phishing-Website-Classifier/training/utilities/webcode/feature_imputer_webcode.pkl",
        )
        print(f"✓ Saved imputer: feature_imputer_webcode.pkl")

    # Save metadata
    metadata = {
        "best_model": best_model_name,
        "best_metrics": results[best_model_name]["metrics"],
        "config": CONFIG,
        "total_models_trained": len(results),
        "imputation_strategy": CONFIG["imputation_strategy"],
    }
    joblib.dump(
        metadata,
        "/home/maliha/Programming/dm/Phishing-Website-Classifier/training/utilities/webcode/model_metadata.pkl",
    )
    print(f"✓ Saved metadata: model_metadata.pkl")

    # Save feature names
    joblib.dump(
        feature_names,
        "/home/maliha/Programming/dm/Phishing-Website-Classifier/training/utilities/webcode/feature_names_webcode.pkl",
    )
    print(f"✓ Saved feature names: feature_names_webcode.pkl")

In [ ]:
print(f"\nConfiguration:")
print(f"  Imputation Strategy: {CONFIG['imputation_strategy'].upper()}")
print(f"  Scaling: {'Enabled' if CONFIG['scale_data'] else 'Disabled'}")
print(
    f"  Imbalance Handling: {'Enabled' if CONFIG['handle_imbalance'] else 'Disabled'}"
)

# Step 1: Load data
df = load_and_explore_data(
    "/home/maliha/Programming/dm/Phishing-Website-Classifier/phishing_complete_features.csv"
)

# Step 2: Prepare data (with advanced imputation)
X_train, X_test, y_train, y_test, scaler, imputer, feature_names = prepare_data(
    df, CONFIG
)

# Step 3: Define models
models = get_models(CONFIG)

# Step 4: Train all models
results = train_all_models(models, X_train, X_test, y_train, y_test, CONFIG)

# Step 5: Compare models
best_model_name, df_comparison = compare_models(results)

# Step 6: Save artifacts
save_artifacts(best_model_name, results, scaler, imputer, feature_names)


Configuration:
  Imputation Strategy: ITERATIVE
  Scaling: Enabled
  Imbalance Handling: Enabled
STEP 1: LOADING DATA
✓ Dataset loaded: (80000, 81)

Data Quality Check:
  Missing values: 0 (0.00%)
  Duplicate rows: 0

Class Distribution:
result
0    50000
1    30000
Name: count, dtype: int64
Phishing ratio: 37.50%
Imbalance ratio: 1.67:1
⚠️  Dataset is imbalanced - will apply SMOTE

STEP 2: PREPARING DATA

Dataset Info:
  Features: 79
  Samples: 80000
  Classes: 2

Data Split:
  Train set: (64000, 79)
  Test set: (16000, 79)

HANDLING MISSING AND INVALID VALUES

2. Checking for missing values...
   ✓ No missing values found

4. Final validation...
   ✓ Data is clean and ready for training

5. Summary:
   Original shape: (64000, 79)
   Final shape: (64000, 79)
   Features preserved: 79

HANDLING MISSING AND INVALID VALUES

2. Checking for missing values...
   ✓ No missing values found

4. Final validation...
   ✓ Data is clean and ready for training

5. Summary:
   Original shape: (160